In [1]:
import os
import pickle
import torch
import sys
sys.path.append("..")
import pandas as pd
from REMI.midi2event import analyzer, corpus, event

In [2]:
# pkl_dict = "../../dataset/Compound_Word_Transformer_Uncond/representations/uncond/remi/ailab17k_from-scratch_remi/dictionary.pkl"
# remi_dictionary = pickle.load(open(pkl_dict, "rb"))
# len(remi_dictionary[0])

In [3]:
path_data_root = "../REMI/ailab17k_from-scratch_remi"
path_dictionary = os.path.join(path_data_root, 'dictionary.pkl')
dictionary = pickle.load(open(path_dictionary, "rb"))
event_to_int_no_key = dictionary[0]
len(event_to_int_no_key)

338

In [14]:
key_profiles = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#','A', 'A#','B','c','c#', 'd', 'd#', 'e', 'f', 'f#', 'g', 'g#', 'a', 'a#', 'b']

In [15]:
event_to_int_key = event_to_int_no_key.copy()

In [20]:
for idx, i in enumerate(key_profiles):
    event_to_int_key["Key_" + i] = 338 + idx

In [21]:
int_to_event = {j:i for i,j in event_to_int_key.items()}

In [22]:
key_dictionary = (event_to_int_key, int_to_event)

In [23]:
path_data_root = "../REMI/ailab17k_from-scratch_remi"
path_dictionary = os.path.join(path_data_root, 'dictionary_with_key.pkl')
pickle.dump(key_dictionary, open(path_dictionary, 'wb'))

In [24]:
len(event_to_int_no_key), len(event_to_int_key)

(338, 362)

# Make REMI_Representation

In [25]:
matlab_path = "../dataset/matlab_feature/"
remi_dir = "../REMI/ailab17k_from-scratch_remi/events"
fnames = os.listdir(remi_dir)

In [30]:
def _get_remi_to_int(fnames, use_key = True):
    for fname in fnames:
        remi_example = pickle.load(open(os.path.join(remi_dir, fname), "rb"))
        fname = fname.split(".")[0].split("_seg")[0]
        if use_key:
            matlab_features = pd.read_csv(os.path.join(matlab_path, fname + ".csv"))
            key_profile = {"name": "Key", "value": matlab_features['keyname'].item()}
            remi_example = [key_profile] + remi_example
            int_example = [event_to_int_key[str(i['name'])+"_"+str(i['value'])] for i in remi_example]
            torch.save(int_example, os.path.join("../dataset/remi_key_midi", fname + ".pt"))
        else:
            int_example = [event_to_int_no_key[str(i['name'])+"_"+str(i['value'])] for i in remi_example]
            torch.save(int_example, os.path.join("../dataset/remi_midi", fname + ".pt"))
    return remi_example

In [31]:
remi_example = _get_remi_to_int(fnames)
# _get_remi_to_int(fnames, use_key=False)

In [32]:
remi_example

[{'name': 'Key', 'value': 'A'},
 {'name': 'Bar', 'value': None},
 {'name': 'Beat', 'value': 0},
 {'name': 'Chord', 'value': 'N_N'},
 {'name': 'Tempo', 'value': 119},
 {'name': 'Beat', 'value': 8},
 {'name': 'Chord', 'value': 'D_sus2'},
 {'name': 'Tempo', 'value': 53},
 {'name': 'Beat', 'value': 11},
 {'name': 'Note_Pitch', 'value': 74},
 {'name': 'Note_Velocity', 'value': 40},
 {'name': 'Note_Duration', 'value': 1920},
 {'name': 'Note_Pitch', 'value': 62},
 {'name': 'Note_Velocity', 'value': 40},
 {'name': 'Note_Duration', 'value': 240},
 {'name': 'Note_Pitch', 'value': 88},
 {'name': 'Note_Velocity', 'value': 58},
 {'name': 'Note_Duration', 'value': 600},
 {'name': 'Beat', 'value': 12},
 {'name': 'Tempo', 'value': 98},
 {'name': 'Bar', 'value': None},
 {'name': 'Beat', 'value': 0},
 {'name': 'Tempo', 'value': 98},
 {'name': 'Note_Pitch', 'value': 88},
 {'name': 'Note_Velocity', 'value': 76},
 {'name': 'Note_Duration', 'value': 720},
 {'name': 'Beat', 'value': 4},
 {'name': 'Tempo', 'v

# Data Split

In [55]:
index_list = []
fname_list = []
label_list = []
for i in fnames:
    index_name = i.split(".mid")[0].split("_seg")[0]
    f_name = "_".join(i.split(".mid")[0].split("_")[1:-1])
    label = i.split("_")[0]
    index_list.append(index_name)
    label_list.append(label)
    fname_list.append(f_name)

In [56]:
df = pd.DataFrame(label_list, index=index_list, columns=['label'])
df.head()

,label
Q3_xYZ8n8ULaNo_0,Q3
Q2_48EYLEAgaBc_1,Q2
Q4_j8Ir-ssM-AA_0,Q4
Q4_I2MrA-o5H8I_2,Q4
Q4_ltNK_MY1HkM_2,Q4


In [88]:
len(df)

1052

In [58]:
from collections import Counter
from sklearn.model_selection import train_test_split

train_val, test = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
len(train_val) ,Counter(train_val['label']), len(test), Counter(test['label'])

(946,
 Counter({'Q2': 236, 'Q3': 219, 'Q1': 215, 'Q4': 276}),
 106,
 Counter({'Q1': 24, 'Q4': 31, 'Q3': 25, 'Q2': 26}))

In [59]:
train, val = train_test_split(train_val, test_size=0.215, random_state=42, stratify=train_val['label'])
len(train) ,Counter(train['label']), len(val), Counter(val['label'])

(742,
 Counter({'Q4': 216, 'Q1': 169, 'Q3': 172, 'Q2': 185}),
 204,
 Counter({'Q4': 60, 'Q1': 46, 'Q2': 51, 'Q3': 47}))

In [63]:
# train.to_csv("../dataset/split/train.csv")
# val.to_csv("../dataset/split/val.csv")
# test.to_csv("../dataset/split/test.csv")

# MIDI to Event

In [4]:
midi_path = "../../dataset/PEmoDataset/midis/"
fname = os.listdir(midi_path)

In [5]:
example = "../dataset/sample_data/example_generative.mid"

In [7]:
midi_obj = analyzer(example)
song_data = corpus(midi_obj)
final_sequence = event(song_data)

In [11]:
song_data.keys()

dict_keys(['notes', 'chords', 'tempos', 'labels', 'metadata'])

In [8]:
final_sequence

[{'name': 'Bar', 'value': None},
 {'name': 'Beat', 'value': 0},
 {'name': 'Chord', 'value': 'N_N'},
 {'name': 'Tempo', 'value': 119},
 {'name': 'Beat', 'value': 2},
 {'name': 'Note_Pitch', 'value': 68},
 {'name': 'Note_Velocity', 'value': 80},
 {'name': 'Note_Duration', 'value': 480},
 {'name': 'Beat', 'value': 4},
 {'name': 'Tempo', 'value': 122},
 {'name': 'Note_Pitch', 'value': 61},
 {'name': 'Note_Velocity', 'value': 90},
 {'name': 'Note_Duration', 'value': 720},
 {'name': 'Note_Pitch', 'value': 61},
 {'name': 'Note_Velocity', 'value': 102},
 {'name': 'Note_Duration', 'value': 720},
 {'name': 'Note_Pitch', 'value': 49},
 {'name': 'Note_Velocity', 'value': 94},
 {'name': 'Note_Duration', 'value': 960},
 {'name': 'Beat', 'value': 6},
 {'name': 'Note_Pitch', 'value': 56},
 {'name': 'Note_Velocity', 'value': 74},
 {'name': 'Note_Duration', 'value': 480},
 {'name': 'Beat', 'value': 8},
 {'name': 'Tempo', 'value': 104},
 {'name': 'Note_Pitch', 'value': 77},
 {'name': 'Note_Velocity', 'va